# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [1]:
'''
# Essentia
!pip install essentia
# Freesound-python
!pip install git+https://github.com/mtg/freesound-python.git
# Mount drive and cd to notebook folder
from google.colab import drive
drive.mount('/content/drive')
'''

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 4.8 MB/s eta 0:00:00
  Cloning https://github.com/mtg/freesound-python.git to /tmp/pip-req-build-mtsluyzr
  Running command git clone --filter=blob:none --quiet https://github.com/mtg/freesound-python.git /tmp/pip-req-build-mtsluyzr
  Resolved https://github.com/mtg/freesound-python.git to commit 5be99a3689d17303c01cb122bbb0d5a96eba04f6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for freesound-python: filename=freesound_python-1.1-py3-none-any.whl size=6714 sha256=73d98a461206e96a43adf935667a7c66d70695b1419e8d84b8459876c683ff4b
  Stored in directory: /tmp/pip-ephem-wheel-cache-_9uczfz0/wheels/2d/14/a0/4f60908e5b801658e71ba0d38958c2e2baef3573ef368db670
Successfully built freesound-python
Mounted at /content/drive


In [5]:
#%cd '/content/drive/MyDrive/SMC/AMPLab2324/AMPLAB 2024 Freesound session'

/content/drive/MyDrive/SMC/AMPLab2324/AMPLAB 2024 Freesound session


In [20]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = '9S69O3BqRfvNCxaKkLi8do7P9AnYzTwNpDIXAFIX'  # Please replace by your own Freesound API key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store
LEAD_SOUNDS_DIR = 'files/lead_sounds'  # Place where to store the lead sounds. Will be relative to the current folder.
LEAD_SOUNDS_METADATA_FILENAME = 'lead_sounds_metadata.csv'  # File where we'll store the metadata of our lead sounds collection
BASS_SOUNDS_DIR = 'files/bass_sounds'  # Place where to store the bass sounds. Will be relative to the current folder.
BASS_SOUNDS_METADATA_FILENAME = 'bass_sounds_metadata.csv'  # File where we'll store the metadata of our bass sounds collection
AMB_SOUNDS_DIR = 'files/amb_sounds'  # Place where to store the amb sounds. Will be relative to the current folder.
AMB_SOUNDS_METADATA_FILENAME = 'amb_sounds_metadata.csv'  # File where we'll store the metadata of our amb sounds collection

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)
if not os.path.exists(LEAD_SOUNDS_DIR): os.mkdir(LEAD_SOUNDS_DIR)
if not os.path.exists(BASS_SOUNDS_DIR): os.mkdir(BASS_SOUNDS_DIR)
if not os.path.exists(AMB_SOUNDS_DIR): os.mkdir(AMB_SOUNDS_DIR)

In [21]:
# Define some util functions

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound, path):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = path + "/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

In [22]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
lead_sound_queries = [
    {
        'query': 'speech',
        'filter': 'duration:[0 TO 2]',
        'num_results': 8,
    },
    {
        'query': 'bird',
        'filter': 'duration:[0 TO 2]',
        'num_results': 16,
    },
    {
        'query': 'singing voice',
        'filter': 'duration:[0 TO 2]',
        'num_results': 16,
    },
    {
        'query': 'whisper',
        'filter': 'duration:[0 TO 2]',
        'num_results': 8,
    },
    {
        'query': 'laugh',
        'filter': 'duration:[0 TO 2]',
        'num_results': 16,
    }
]

# Do all queries and concatenate the results in a single list of sounds
lead_sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in lead_sound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(lead_sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(lead_sounds)))
    retrieve_sound_preview(sound, LEAD_SOUNDS_DIR)

# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s, LEAD_SOUNDS_DIR) for s in lead_sounds])
df.to_csv(LEAD_SOUNDS_METADATA_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), LEAD_SOUNDS_METADATA_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 64 entries! lead_sounds_metadata.csv


,name,username,license,tags,freesound_id,path
0,Atari Speech.wav,mo_damage,http://creativecommons.org/publicdomain/zero/1.0/,"[8-bit, atari, engine, speech, steem, texttosp...",100981,files/lead_sounds/100981_1234256-hq.ogg
1,Speech_Signal_Processing_Male.wav,dpsa,http://creativecommons.org/licenses/by/3.0/,"[DPSA, Male, Speech]",341507,files/lead_sounds/341507_5871007-hq.ogg
2,americanm2_4.mp3,kwahmah_02,http://creativecommons.org/licenses/by/3.0/,"[voice, number, text-to-speech, american, spee...",256886,files/lead_sounds/256886_4486188-hq.ogg
3,areyouready1.wav,irrlicht,https://creativecommons.org/licenses/by/4.0/,"[1-bit, lo-fi, rave, retro, speech, synthesize...",121231,files/lead_sounds/121231_81909-hq.ogg
4,exterminate.wav,Timbre,http://creativecommons.org/publicdomain/zero/1.0/,"[robotic, computer, speech, Atari, exterminate]",420717,files/lead_sounds/420717_1015240-hq.ogg
...,...,...,...,...,...,...
59,laugh5.wav,Reitanna,http://creativecommons.org/publicdomain/zero/1.0/,"[woman, laugh, female, english, girl, voice, t...",241500,files/lead_sounds/241500_950925-hq.ogg
60,laugh oh god3.wav,Reitanna,http://creativecommons.org/publicdomain/zero/1.0/,"[woman, laugh, female, english, girl, voice, t...",241505,files/lead_sounds/241505_950925-hq.ogg
61,that laugh that hurts.wav,Reitanna,http://creativecommons.org/publicdomain/zero/1.0/,"[woman, laugh, female, english, girl, voice, t...",241519,files/lead_sounds/241519_950925-hq.ogg
62,laugh7.wav,Reitanna,http://creativecommons.org/publicdomain/zero/1.0/,"[woman, laugh, female, english, girl, voice, t...",241520,files/lead_sounds/241520_950925-hq.ogg


In [23]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
bass_sound_queries = [
    {
        'query': 'machine hum',
        'filter': 'duration:[1 TO 4]',
        'num_results': 20,
    },
    {
        'query': 'ship foghorn',
        'filter': 'duration:[1 TO 4]',
        'num_results': 16,
    },
    {
        'query': 'industry',
        'filter': 'duration:[1 TO 4]',
        'num_results': 8,
    },
    {
        'query': 'machine gears',
        'filter': 'duration:[1 TO 4]',
        'num_results': 16,
    },
    {
        'query': 'boom',
        'filter': 'duration:[2 TO 8]',
        'num_results': 16,
    }
]

# Do all queries and concatenate the results in a single list of sounds
bass_sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in bass_sound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(bass_sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(bass_sounds)))
    retrieve_sound_preview(sound, BASS_SOUNDS_DIR)

# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s, BASS_SOUNDS_DIR) for s in bass_sounds])
df.to_csv(BASS_SOUNDS_METADATA_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), BASS_SOUNDS_METADATA_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 62 entries! bass_sounds_metadata.csv


,name,username,license,tags,freesound_id,path
0,coke machine hum.wav,cognito perceptu,http://creativecommons.org/publicdomain/zero/1.0/,"[drink, drone, hum, machine, pop, soda]",17011,files/bass_sounds/17011_57789-hq.ogg
1,Sewing machine sound processed,Jagadamba,https://creativecommons.org/licenses/by/4.0/,"[MACHINE, industrial, thread, Hum, sewing, fac...",257023,files/bass_sounds/257023_4062622-hq.ogg
2,electroHum.aiff,fer.scope,https://creativecommons.org/licenses/by-nc/4.0/,"[machine, hum, noise]",185837,files/bass_sounds/185837_967820-hq.ogg
3,machineair.wav,MetaKnight11,http://creativecommons.org/publicdomain/zero/1.0/,"[air, airy, audacity, hum, machine, synthetic,...",72106,files/bass_sounds/72106_448953-hq.ogg
4,Rumble_01,oceansonmars,http://creativecommons.org/publicdomain/zero/1.0/,"[Electric, Voice, Static, Machine, Radio, Buzz...",717119,files/bass_sounds/717119_15225418-hq.ogg
...,...,...,...,...,...,...
57,field recording backyard.thunder.shed.tap.boo...,thebraindrinker,http://creativecommons.org/publicdomain/zero/1.0/,"[shed, tap, thunder, boom, wav, backyard]",397265,files/bass_sounds/397265_4704538-hq.ogg
58,Fireworks boom far 3,MaoDin204,http://creativecommons.org/publicdomain/zero/1.0/,"[firework, boom, Fireworks]",722270,files/bass_sounds/722270_15555277-hq.ogg
59,Firework boom with crackle 2,MaoDin204,http://creativecommons.org/publicdomain/zero/1.0/,"[firework, boom, Fireworks]",722273,files/bass_sounds/722273_15555277-hq.ogg
60,Firework boom with crackle 10,MaoDin204,http://creativecommons.org/publicdomain/zero/1.0/,"[Fireworks, boom, firework]",722288,files/bass_sounds/722288_15555277-hq.ogg


In [24]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
amb_sound_queries = [
    {
        'query': 'village',
        'filter': 'duration:[5 TO 10]',
        'num_results': 4,
    },
    {
        'query': 'rain',
        'filter': 'duration:[5 TO 10]',
        'num_results': 4,
    },
    {
        'query': 'forest',
        'filter': 'duration:[5 TO 10]',
        'num_results': 4,
    },
    {
        'query': 'traffic',
        'filter': 'duration:[5 TO 10]',
        'num_results': 4,
    },
    {
        'query': 'ocean',
        'filter': 'duration:[5 TO 10]',
        'num_results': 4,
    }
]

# Do all queries and concatenate the results in a single list of sounds
amb_sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in amb_sound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(amb_sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(amb_sounds)))
    retrieve_sound_preview(sound, AMB_SOUNDS_DIR)

# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s, AMB_SOUNDS_DIR) for s in amb_sounds])
df.to_csv(AMB_SOUNDS_METADATA_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), AMB_SOUNDS_METADATA_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 20 entries! amb_sounds_metadata.csv


,name,username,license,tags,freesound_id,path
0,Stream.wav,14FPanskaSilovsky_Petr,http://creativecommons.org/publicdomain/zero/1.0/,"[Pansk, Stream, Czech, CZ, Panska, Village]",419929,files/amb_sounds/419929_8373708-hq.ogg
1,"A donkey is braying in the field, close to the...",felix.blume,http://creativecommons.org/publicdomain/zero/1.0/,"[donkey, blume, mexico, bray, felix, bawl, pre...",133761,files/amb_sounds/133761_1661766-hq.ogg
2,Donkey braying loud with distant village atmos...,felix.blume,http://creativecommons.org/publicdomain/zero/1.0/,"[donkey, screaming, village, animal, loud, Fel...",705300,files/amb_sounds/705300_1661766-hq.ogg
3,001-auto.wav,13GPanska_Dubnova_Magdalena,http://creativecommons.org/publicdomain/zero/1.0/,"[car, field-recording, village]",379165,files/amb_sounds/379165_7027631-hq.ogg
4,RAIN STICK B 004.wav,sandyrb,https://creativecommons.org/licenses/by/4.0/,"[chile, chilean, ghana, ghanaian, instrument, ...",86361,files/amb_sounds/86361_14771-hq.ogg
5,AMBIENT - Rain - Light - Near Drainpipe (LOOP)...,Arctura,http://creativecommons.org/licenses/by/3.0/,"[deep, drainpipe, droplets, field-recording, n...",34066,files/amb_sounds/34066_28216-hq.ogg
6,Rain-Heavy-Loopable.wav,svampen,http://creativecommons.org/licenses/by/3.0/,"[weather, loop, rain, field-recording]",334149,files/amb_sounds/334149_5910095-hq.ogg
7,room-tone wind rain 11 200216_0114.wav,klankbeeld,https://creativecommons.org/licenses/by/4.0/,"[raining, atmosphere, general-noise, house, ga...",522752,files/amb_sounds/522752_1648170-hq.ogg
8,walk-forest02.wav,JanKoehl,https://creativecommons.org/licenses/by/4.0/,"[branch, crack, forest, ground, step, stomp, w...",85602,files/amb_sounds/85602_1078663-hq.ogg
9,birds in the forest,13F_Panska_Petruzelova_Anna,http://creativecommons.org/publicdomain/zero/1.0/,"[atmosphere, forest, birds]",379246,files/amb_sounds/379246_7027306-hq.ogg
